# Note: This code is experimental. 

### These are some attempts to get a CNN model working with healpix coordinates. 

## CNN on healpix model (not working):

In [ ]:
import os
import time
import torch
import pickle
from src.cone_model import ToyModel3DCone, HEALPixCone
from src.loss import ApproxLoss
from src.model import ApproxModel
from src.config import Config
from src.train import TrainerMain as Trainer
from src.utils import set_seed, ensure_dir_exists

import nnhealpix
import tensorflow as tf

import numpy as np

if __name__ == '__main__':

    # Config
    config = Config()
    config.model_type = 'conv'
    config.loss_type = 'MSELoss'
    config.metric_monitor = 'loss'
    config.lr = 1e-3
    config.dropout_rate = 0.0
    config.train_batch_size = 1024 # number of inputs to process before updating the model (backpropagation). PREV 1024
    config.eval_batch_size = 1024 # PREV 1024
    config.epoch = 5000
    config.device = 'cpu'
    
    
    # flattened = True for using fully-connected layers, False for using conv-based layers
    config.flattened = False  
    config.filter_size = 3
    config.NSIDE = 8 # number of constant latitude rings. larger means significantly more parameters. PREV 6. 
    config.exp_name = 'sphere_{}_{}_NSIDE{}_datasize1024'.format(
       config.model_type, config.loss_type, config.NSIDE)
    config.working_dir = os.path.join('results', 
        '{}_{}'.format(config.exp_name, time.strftime('%m%d_%H-%M'))
    )
    ensure_dir_exists(config.working_dir)
    config.dump(os.path.join(config.working_dir, 'config.json'))
    

    set_seed(2021)

In [ ]:
'''
    
Generation and saving of the dataset.

'''

cone_model = HEALPixCone(
        output_dir=os.path.join(config.working_dir, 'figs'),
        NSIDE=config.NSIDE
)

train_dset = cone_model.create_dataset(dataset_size=1024) 
val_dset = cone_model.create_dataset(dataset_size=1024)   
f = open(f"sphere_datasets_NSIDE{config.NSIDE}.pkl", "wb")
pickle.dump((train_dset, val_dset), f)
f.close() 


In [ ]:
'''
Open the dataset.
'''

f = open(f"sphere_datasets_NSIDE{config.NSIDE}.pkl", "rb")
train_dset, val_dset = pickle.load(f)
f.close()


In [ ]:
'''

Create the training and validation datasets. Do any necessary reshapping. 

'''

x_train = []
y_train = []

x_val = []
y_val = []

# versions of y_train/y_proj that aren't just flattened. Meaning you have 4 separate images (or however many there are based on depth) that you can project onto the globe.
y_train_proj = [] 
y_val_proj = []


for i in range(len(train_dset)):
    x_train.append(train_dset[i]['data'].reshape(2,1))  # can index like dictionary to get ith data entry (data, label) pair
    x_val.append(val_dset[i]['data'].reshape(2,1))

    # flattened version
    y_train.append(train_dset[i]['label'])
    y_val.append(val_dset[i]['label'])

    # non flattened version
    # y_train.append(val_dset[i]['label'].reshape(768, 4)) # 4 is depth. second # is length of train_dset / depth which is used for size of each healpix globe thing (# pixels) 
    # y_val.append(val_dset[i]['label'].reshape(768, 4))

    # add versions of y_train/val that can be projected onto the sphere
    y_train_proj.append(val_dset[i]['label'].reshape(4, 768)) # 4 is depth. second # is length of train_dset / depth which is used for size of each healpix globe thing (# pixels). will take first 768 then start new row.
    y_val_proj.append(val_dset[i]['label'].reshape(4, 768))

x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)

y_train_proj = np.array(y_train_proj)
y_val_proj = np.array(y_val_proj)

print("x train dim: ", x_train.shape)
print("y train dim: ", y_train.shape)
print()

print("x val dim: ", x_val.shape)
print("y val dim: ", y_val.shape)

print()

print("y projection version dim: ", y_train_proj.shape)

In [ ]:
'''

Make image of one of the data points to make sure the file reading/reshaping is correct.

'''


import healpy as hp
import matplotlib.pyplot as plt


hp.projview(
    y_train_proj[25][2], # first number is which datapoint and second is which cone depth (1-4)
    coord=["G"],
    graticule=True,
    graticule_labels=True,
    unit="cbar label",
    xlabel="longitude",
    ylabel="latitude",
    cb_orientation="horizontal",
    projection_type="mollweide",
)
plt.savefig("mygraph.png")


In [ ]:

'''

Create the model.

'''

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, activations
import nnhealpix.layers



inputs = layers.Input(shape=(2, 1)) # inputs of the convolution part
input_NSIDE = config.NSIDE

# I will be using the functional API for keras (functions to make model) since more flexible than sequential.

# # blow it up
conv = layers.Dense(3072)(inputs)
conv = layers.Activation('relu')(conv) 

# add convolutions
conv = nnhealpix.layers.ConvNeighbours(input_NSIDE, filters=32, kernel_size=9)(inputs)  # i think filters should be multple of 3072
conv = layers.Activation('relu')(conv) # activation after every convolution

# conv = nnhealpix.layers.ConvNeighbours(input_NSIDE, filters=32, kernel_size=9)(conv)
# conv = tf.keras.layers.Activation('relu')(conv) # activation after every convolution

# conv = nnhealpix.layers.ConvNeighbours(input_NSIDE, filters=4, kernel_size=9)(conv)
# conv = tf.keras.layers.Activation('relu')(conv) # activation after every convolution


# # add a maxpooling layer
# conv = nnhealpix.layers.MaxPooling(input_NSIDE, 4)(conv) # for maxpooling params: (NSIDE_IN, NSIDE_OUT). NSIDE_OUT must be lower power of 2 based on files in library.
# conv = tf.keras.layers.Dropout(0.2)(conv) # add dropout to prevent overfitting

# flatten to pass to next part!
conv = layers.Flatten()(conv)

# # regular fully connected part
# conv = tf.keras.layers.Dense(1024)(conv)
# conv = tf.keras.layers.Activation('relu')(conv)

# conv = tf.keras.layers.Dense(2048)(conv)
# conv = tf.keras.layers.Activation('relu')(conv)

conv = layers.Dense(3072)(conv) # 768 * 4
out = layers.Activation('softmax')(conv)

model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(loss=tf.keras.losses.mse, optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=1000, validation_data=(x_val, y_val))

## Regular Fully Connected Network with Healpix (works).

In [ ]:
'''

Unlike above, the x train will be just lists of 2 for each data point (not 2, 1)

'''


x_train = []
y_train = []

x_val = []
y_val = []

# versions of y_train/y_proj that aren't just flattened. Meaning you have 4 separate images (or however many there are based on depth) that you can project onto the globe.
y_train_proj = [] 
y_val_proj = []


for i in range(len(train_dset)):
    x_train.append(train_dset[i]['data'])  # can index like dictionary to get ith data entry (data, label) pair
    x_val.append(val_dset[i]['data'])

    # flattened version
    y_train.append(train_dset[i]['label'])
    y_val.append(val_dset[i]['label'])

    # non flattened version
    # y_train.append(val_dset[i]['label'].reshape(768, 4)) # 4 is depth. second # is length of train_dset / depth which is used for size of each healpix globe thing (# pixels) 
    # y_val.append(val_dset[i]['label'].reshape(768, 4))

    # add versions of y_train/val that can be projected onto the globe
    y_train_proj.append(val_dset[i]['label'].reshape(4, 768)) # 4 is depth. second # is length of train_dset / depth which is used for size of each healpix globe thing (# pixels). will take first 768 then start new row.
    y_val_proj.append(val_dset[i]['label'].reshape(4, 768))

x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)

y_train_proj = np.array(y_train_proj)
y_val_proj = np.array(y_val_proj)

print("x train dim: ", x_train.shape)
print("y train dim: ", y_train.shape)
print()

print("x val dim: ", x_val.shape)
print("y val dim: ", y_val.shape)

print()

print("y projection version dim: ", y_train_proj.shape)

In [ ]:

'''

Create the model.

'''

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, activations
import nnhealpix.layers


'''

Working FCNN. 

'''

inputs = layers.Input(shape=(2, )) # inputs of the convolution part
input_NSIDE = config.NSIDE

# I will be using the functional API for keras (functions to make model) since more flexible than sequential.

conv = layers.Dense(10)(inputs)
conv = layers.Activation('relu')(conv) 

conv = layers.Dense(1000)(conv)
conv = layers.Activation('relu')(conv) 

# conv = layers.Dense(1000)(conv)
# conv = layers.Activation('relu')(conv) 

conv = layers.Dense(1000)(conv)
conv = layers.Activation('relu')(conv) 

out = layers.Dense(3072)(conv) # 768 * 4
# out = layers.Activation('softmax')(conv)

model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(loss=tf.keras.losses.mse, optimizer=tf.keras.optimizers.Adam(lr=0.01)) 
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=200, validation_data=(x_val, y_val))

In [ ]:
pred = model.predict(x_train)

one_pred = pred[35].reshape(4, 768)


import healpy as hp
import matplotlib.pyplot as plt


hp.projview(
    one_pred[2], # first number is which datapoint and second is which cone depth (1-4)
    coord=["G"],
    graticule=True,
    graticule_labels=True,
    unit="cbar label",
    xlabel="longitude",
    ylabel="latitude",
    cb_orientation="horizontal",
    projection_type="mollweide",
)
plt.savefig("mygraph.png")


## TRAINING CNN BASED ON OUTPUT OF FULLY CONNECTED (failed): 

Loss was not reducing. I tried to restrict it to only one depth of the cone that it was predicting but that didn't work.

In [ ]:
# get predictions for training set and val set

# train_pred = model.predict(x_train)
# val_pred = model.predict(x_val)

# reshape to be (1024 by 3072 by 1) instead of just 1024 by 3072. this is how the conv layer wants it.

train_pred = train_pred.reshape(1024, 4, 768, 1);
val_pred = val_pred.reshape(1024, 4, 768, 1);


In [ ]:
# lets only get one depth of it
cnn_x_train = []
cnn_x_val = []

# lets only get one depth of it
cnn_y_train = []
cnn_y_val = []

# y trains from beginning but reshaped  with onyl one depth
temp_y_train = y_train.reshape(1024, 4, 768)
temp_y_val = y_val.reshape(1024, 4, 768)


depth = 2

for i in range(len(train_pred)):
    cnn_x_train.append(train_pred[i][depth])
    cnn_y_train.append(temp_y_train[i][depth])

for i in range(len(val_pred)):
    cnn_x_val.append(val_pred[i][depth])
    cnn_y_val.append(temp_y_val[i][depth])

cnn_x_train = np.array(cnn_x_train)
cnn_x_val = np.array(cnn_x_val)


cnn_y_train = np.array(cnn_y_train)
cnn_y_val = np.array(cnn_y_val)

print("cnn x train shape: ", cnn_x_train.shape)
print("cnn y train shape: ", cnn_y_train.shape)

print("cnn x val shape: ", cnn_x_val.shape)
print("cnn y val shape: ", cnn_y_val.shape)


In [ ]:
'''

Create the model.

'''


import tensorflow as tf
from tensorflow.keras import datasets, layers, models, activations
import nnhealpix.layers


inputs = layers.Input(shape=(768, 1)) # inputs of the convolution part
input_NSIDE = config.NSIDE

# add convolutions
cnn = nnhealpix.layers.ConvNeighbours(input_NSIDE, filters=40, kernel_size=9)(inputs)  # i think filters should be multple of 3072
cnn = layers.Activation('relu')(cnn) # activation after every convolution

cnn = nnhealpix.layers.ConvNeighbours(input_NSIDE, filters=40, kernel_size=9)(cnn)
cnn = tf.keras.layers.Activation('relu')(cnn) # activation after every convolution


# # add a maxpooling layer
# cnn = nnhealpix.layers.MaxPooling(input_NSIDE, 4)(cnn) # for maxpooling params: (NSIDE_IN, NSIDE_OUT). NSIDE_OUT must be lower power of 2 based on files in library.
# cnn = tf.keras.layers.Dropout(0.2)(cnn) # add dropout to prevent overfitting

# flatten to pass to next part!
cnn = layers.Flatten()(cnn)

cnn = layers.Dense(768)(cnn) # 768 * 4
out = layers.Activation('softmax')(cnn)

model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(loss=tf.keras.losses.mse, optimizer=tf.keras.optimizers.Adam(lr=0.01), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(cnn_x_train, cnn_y_train, epochs=1000, validation_data=(cnn_x_val, cnn_y_val))